# Probabilities through monte carlo simulations

Here, rather than training a model to predict the $E[\text{dice roll score}]$, I just build a monte carlo simulation of a Farkle turn. 

In [1]:
import sys
sys.path.insert(0, '../../src')

In [2]:
import random
from collections import defaultdict

from farkle.logic import gameobjects as go

In [3]:
dh1 = go.DiceHand()

In [4]:
dh1

0: 5
1: 6
2: 4
3: 1
4: 6
5: 3
Score: 0

In [13]:
class ExpectedRollScore():
    def __init__(self, num_dice: int):
        self.num_dice = num_dice
        self.obs = 0
        self.possible_score_counts = defaultdict(int)
    
    def score_frequ(self):
        dh: go.DiceHand
        sf = {dh: c / self.obs for dh, c in self.possible_score_counts.items()}
        return sf
    
    def count_score(self, dh: go.DiceHand):
        assert dh.num_dice <= self.num_dice
        self.obs += 1
        self.possible_score_counts[dh] += 1
        
    def count_farkle(self):
        self.obs += 1
        self.possible_score_counts['farkle'] += 1
        
    def expected_score(self, points_at_risk: int) -> int:
        dh: go.DiceHand
        s = sum([dh.score * prob if dh != 'farkle' 
                 else -points_at_risk * prob 
                 for dh, prob in self.score_frequ().items()])
        return s
        
    def __repr__(self):
        return f'ExpectedRollScore(num_dice={self.num_dice})'

In [14]:
ers = ExpectedRollScore(6)

In [15]:
ers.count_score(dh1)

In [16]:
ers.count_score(go.DiceHand(1,1,1, score=1000))

In [17]:
ers.count_farkle()

In [18]:
ers.count_farkle()

In [19]:
ers.score_frequ()

{0: 5
 1: 6
 2: 4
 3: 1
 4: 6
 5: 3
 Score: 0: 0.25,
 0: 1
 1: 1
 2: 1
 Score: 1000: 0.25,
 'farkle': 0.5}

In [20]:
ers.expected_score(200)

150.0

In [76]:
class StatTracker():
    def __init__(self):
        self.epoch = 0
        
        self.dice_to_roll_probs = {i+1: ExpectedRollScore(i+1) for i in range(6)}
        
    def expected_score(self, num_dice: int, points_at_risk: int, depth: int = 5) -> float:
        """
        num_dice: number of dice to roll
        points_at_risk: points lost if Farkle
        depth: number of recursive calls allowed
        """
        ers = self.dice_to_roll
    
    @property
    def expected_hot_dice_score(self) -> float:
        return self.expected_score(num_dice=6)

In [77]:
st = StatTracker()
st.dice_to_roll_probs

{1: ExpectedRollScore(num_dice=1),
 2: ExpectedRollScore(num_dice=2),
 3: ExpectedRollScore(num_dice=3),
 4: ExpectedRollScore(num_dice=4),
 5: ExpectedRollScore(num_dice=5),
 6: ExpectedRollScore(num_dice=6)}